In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`

import $ivy.$

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

## Start

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Dataframe API")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 16:22:34 INFO SparkContext: Running Spark version 3.5.5
25/03/13 16:22:34 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 16:22:34 INFO SparkContext: Java version 11.0.26
25/03/13 16:22:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark: SparkSession = org.apache.spark.sql.SparkSession@23c9b624
import spark.implicits._

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.5.5


## 1 Создание DataFrame

### createDataFrame

#### From List

In [6]:
val data1 = Seq(("1", "Spark"), ("2", "Scala"), ("3", "Java"))

data1: Seq[(String, String)] = List(
  ("1", "Spark"),
  ("2", "Scala"),
  ("3", "Java")
)

In [7]:
val df1 = spark.createDataFrame(data1)
df1.show

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



df1: DataFrame = [_1: string, _2: string]

In [8]:
val df1WithNames = df1.withColumnsRenamed(Map("_1" -> "StudentID", "_2" -> "Course"))
df1WithNames.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



df1WithNames: DataFrame = [StudentID: string, Course: string]

##### withSchema

In [9]:
val schema1 = StructType(Array(StructField("StudentID", StringType, false),
                              StructField("Course", StringType, true)))

val schema2 = new StructType()
                    .add(StructField("StudentID", StringType, false))
                    .add(StructField("Course", StringType, true))

val schema3 = StructType(
                StructField("StudentID", StringType, false) :: 
                StructField("Course", StringType, true) :: 
                Nil)

schema1.equals(schema2)
schema2.equals(schema3)

schema1: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
schema2: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
schema3: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
res9_3: Boolean = true
res9_4: Boolean = true

In [10]:
import collection.JavaConverters._

val data2 = data1.map(s => Row(s._1, s._2)).asJava
val df2 = spark.createDataFrame(data2, schema2)
df2.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



import collection.JavaConverters._
data2: java.util.List[Row] = SeqWrapper(List([1,Spark], [2,Scala], [3,Java]))
df2: DataFrame = [StudentID: string, Course: string]

#### fromRDD

In [11]:
val rdd1 = spark.sparkContext.parallelize(data1)
val fromRDD1 = spark.createDataFrame(rdd1)
fromRDD1.show

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



rdd1: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[0] at parallelize at cmd11.sc:1
fromRDD1: DataFrame = [_1: string, _2: string]

##### withSchema

In [12]:
val rdd2 = rdd1.map(s => Row(s._1, s._2))
val fromRDD2 = spark.createDataFrame(rdd2, schema2)
fromRDD2.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



rdd2: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[4] at map at cmd12.sc:1
fromRDD2: DataFrame = [StudentID: string, Course: string]

### toDF

#### From List

In [13]:
val df11 = data1.toDF()
df11.show

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



df11: DataFrame = [_1: string, _2: string]

In [14]:
data1.toDF("StudentID", "Course").show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



In [15]:
val columns = List("StudentID", "Course")
val df12 = data1.toDF(columns: _*)
df12.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



columns: List[String] = List("StudentID", "Course")
df12: DataFrame = [StudentID: string, Course: string]

#### fromRDD

In [16]:
val fromRDD11 = rdd1.toDF()
fromRDD11.show

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



fromRDD11: DataFrame = [_1: string, _2: string]

In [17]:
val fromRDD12 = rdd1.toDF(columns: _*)
fromRDD12.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



fromRDD12: DataFrame = [StudentID: string, Course: string]

### fromFile

In [18]:
val fromFile1 = spark.read.format("json").load("data/customer_data.json")
fromFile1.show(5, false)

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

fromFile1: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [19]:
fromFile1.printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



In [20]:
val fileSchema = StructType(
                    StructField("Address", StringType, true) ::
                    StructField("Birthdate", StringType, true) ::
                    StructField("Country", StringType, true) ::
                    StructField("CustomerID", StringType, true) ::
                    StructField("Email", StringType, true) ::
                    StructField("Name", StringType, true) ::
                    StructField("Username", StringType, true) ::
                    Nil)

fileSchema: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Birthdate", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)

In [21]:
val fromFile2 = spark.read.format("json").schema(fileSchema).load("data/customer_data.json")
fromFile2.show(5, false)

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

fromFile2: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

## 2 Основные операции

In [22]:
val customerDf = fromFile1

customerDf: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [23]:
customerDf.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



In [24]:
customerDf.schema

res24: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Birthdate", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)

In [25]:
customerDf.dtypes

res25: Array[(String, String)] = Array(
  ("Address", "StringType"),
  ("Birthdate", "StringType"),
  ("Country", "StringType"),
  ("CustomerID", "StringType"),
  ("Email", "StringType"),
  ("Name", "StringType"),
  ("Username", "StringType")
)

In [26]:
val h = customerDf.head

h: Row = [Unit 1047 Box 4089
DPO AA 57348,1994-02-20 00:46:27,United Kingdom,12346,cooperalexis@hotmail.com,Lindsay Cowan,valenciajennifer]

In [27]:
h(1)

res27: Any = "1994-02-20 00:46:27"

#### Select

In [28]:
customerDf.select("Birthdate", "Country").show(10)

+-------------------+--------------+
|          Birthdate|       Country|
+-------------------+--------------+
|1994-02-20 00:46:27|United Kingdom|
|1988-06-21 00:15:34|       Iceland|
|1974-11-26 15:30:20|       Finland|
|1977-05-06 23:57:35|         Italy|
|1996-09-13 19:14:27|        Norway|
|1969-06-21 03:39:20|        Norway|
|1993-02-25 18:37:29|       Bahrain|
|1993-03-13 12:37:34|         Spain|
|1972-11-10 12:01:08|       Bahrain|
|1973-01-13 17:17:26|      Portugal|
+-------------------+--------------+
only showing top 10 rows



In [29]:
customerDf.select(col("Country")).show(10)

+--------------+
|       Country|
+--------------+
|United Kingdom|
|       Iceland|
|       Finland|
|         Italy|
|        Norway|
|        Norway|
|       Bahrain|
|         Spain|
|       Bahrain|
|      Portugal|
+--------------+
only showing top 10 rows



In [30]:
customerDf.selectExpr("Birthdate as Date").show(10)

+-------------------+
|               Date|
+-------------------+
|1994-02-20 00:46:27|
|1988-06-21 00:15:34|
|1974-11-26 15:30:20|
|1977-05-06 23:57:35|
|1996-09-13 19:14:27|
|1969-06-21 03:39:20|
|1993-02-25 18:37:29|
|1993-03-13 12:37:34|
|1972-11-10 12:01:08|
|1973-01-13 17:17:26|
+-------------------+
only showing top 10 rows



In [31]:
customerDf.withColumn("Flag", lit(true)).show(10)

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+----+
|             Address|          Birthdate|       Country|CustomerID|               Email|             Name|        Username|Flag|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+----+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|true|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|true|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|true|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|true|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com| 

In [32]:
val dfRenamed = customerDf.withColumnRenamed("Birthdate", "Date")
dfRenamed.show(10)

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|             Address|               Date|       Country|CustomerID|               Email|             Name|        Username|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com|     Natalie Ford| gregoryharrison|


dfRenamed: DataFrame = [Address: string, Date: string ... 5 more fields]

In [33]:
customerDf.schema
dfRenamed.schema

res33_0: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Birthdate", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)
res33_1: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Date", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)

#### Filter

In [34]:
customerDf.filter("Country = 'Norway'").show(10)

+--------------------+-------------------+-------+----------+--------------------+----------------+---------------+
|             Address|          Birthdate|Country|CustomerID|               Email|            Name|       Username|
+--------------------+-------------------+-------+----------+--------------------+----------------+---------------+
|17677 Mark Crest\...|1996-09-13 19:14:27| Norway|     12350|amyholland@yahoo.com|    Natalie Ford|gregoryharrison|
|50047 Smith Point...|1969-06-21 03:39:20| Norway|     12352| vcarter@hotmail.com|     Dana Clarke|         hmyers|
|0297 Jacob Ranch ...|1990-06-01 14:49:52| Norway|     12381|douglaschavez@hot...|   Matthew Jones|    stephanie68|
|3102 Hopkins Walk...|1976-06-19 08:10:24| Norway|     12430|crystalromero@hot...|       Lisa Tate|       pgilbert|
|637 Philip Lock S...|1984-06-06 09:36:14| Norway|     12432|jessica87@hotmail...|  Cheryl Herring|mathewsnicholas|
|546 Tyler Prairie...|1985-05-27 10:39:47| Norway|     12433|mariahmcphe

In [35]:
customerDf.where($"Country" =!= "Iceland").show(10, false)

+-----------------------------------------------------+-------------------+--------------+----------+------------------------+-----------------+----------------+
|Address                                              |Birthdate          |Country       |CustomerID|Email                   |Name             |Username        |
+-----------------------------------------------------+-------------------+--------------+----------+------------------------+-----------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                     |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan    |valenciajennifer|
|Unit 2676 Box 9352\nDPO AA 38560                     |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez  |serranobrian    |
|2765 Powers Meadow\nHeatherfurt, CT 53165            |1977-05-06 23:57:35|Italy         |12349     |dustin37@yahoo.com      |Brad Cardenas    |charleshudson   |
|17677 Mark Crest\nWalterber

#### Sort

In [36]:
customerDf.sort(col("CustomerID").desc).show(10)

+--------------------+-------------------+--------------+----------+--------------------+--------------------+-----------------+
|             Address|          Birthdate|       Country|CustomerID|               Email|                Name|         Username|
+--------------------+-------------------+--------------+----------+--------------------+--------------------+-----------------+
|6942 Connie Skywa...|1973-10-24 00:52:10|United Kingdom|     12989| amber97@hotmail.com|   Brandon Contreras|           ecasey|
|79375 David Neck\...|1971-05-04 22:20:10|United Kingdom|     12988|   erica98@gmail.com|      Gabriel Romero|          qknight|
|00881 West Flat\n...|1997-03-05 19:20:57|United Kingdom|     12987|    vkeith@yahoo.com|Christopher Lawrence|        smcintyre|
|499 Jonathan Stre...|1987-10-24 20:05:15|United Kingdom|     12985| fredsmith@yahoo.com|        Xavier Myers|stricklandjeffery|
|9505 Melissa Stre...|1975-09-22 15:21:58|United Kingdom|     12984|scottjonathan@yah...|        

In [37]:
customerDf.orderBy("CustomerID").show(10)

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|             Address|          Birthdate|       Country|CustomerID|               Email|             Name|        Username|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com|     Natalie Ford| gregoryharrison|


#### Repartition

In [38]:
val repartitionedDf = customerDf.repartition(5, col("Country"))
println(s"Old num partitions: ${customerDf.rdd.getNumPartitions}")
println(s"New num partitions: ${repartitionedDf.rdd.getNumPartitions}")

Old num partitions: 1
New num partitions: 5


repartitionedDf: Dataset[Row] = [Address: string, Birthdate: string ... 5 more fields]

In [39]:
println(s"Num partitions after coalesce: ${repartitionedDf.coalesce(1).rdd.getNumPartitions}")

Num partitions after coalesce: 1


### functions

In [40]:
val datDf = customerDf.select($"Birthdate", date_format(col("Birthdate"), "yyyy-MM-dd").alias("bd"))
datDf.show(5)

+-------------------+----------+
|          Birthdate|        bd|
+-------------------+----------+
|1994-02-20 00:46:27|1994-02-20|
|1988-06-21 00:15:34|1988-06-21|
|1974-11-26 15:30:20|1974-11-26|
|1977-05-06 23:57:35|1977-05-06|
|1996-09-13 19:14:27|1996-09-13|
+-------------------+----------+
only showing top 5 rows



datDf: DataFrame = [Birthdate: string, bd: string]

In [41]:
datDf.printSchema

root
 |-- Birthdate: string (nullable = true)
 |-- bd: string (nullable = true)



In [42]:
customerDf.withColumn("Identity", array($"Name", $"Username")).printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- Identity: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [43]:
customerDf
    .withColumn("Identity", array($"Name", $"Username"))
    .select("Name", "Username", "Identity")
    .show(5, false)

+---------------+----------------+---------------------------------+
|Name           |Username        |Identity                         |
+---------------+----------------+---------------------------------+
|Lindsay Cowan  |valenciajennifer|[Lindsay Cowan, valenciajennifer]|
|Katherine David|hillrachel      |[Katherine David, hillrachel]    |
|Leslie Martinez|serranobrian    |[Leslie Martinez, serranobrian]  |
|Brad Cardenas  |charleshudson   |[Brad Cardenas, charleshudson]   |
|Natalie Ford   |gregoryharrison |[Natalie Ford, gregoryharrison]  |
+---------------+----------------+---------------------------------+
only showing top 5 rows



## 3 Агрегирование

In [44]:
customerDf.groupBy("Country").count().orderBy("count").show(10)

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|              Brazil|    1|
|             Iceland|    1|
|United Arab Emirates|    1|
|      Czech Republic|    1|
|           Singapore|    1|
|                 RSA|    1|
|        Saudi Arabia|    1|
|             Lebanon|    1|
|              Greece|    2|
|         Unspecified|    2|
+--------------------+-----+
only showing top 10 rows



In [45]:
customerDf
    .groupBy($"Country", date_format(col("Birthdate"), "yyyy-MM-dd").alias("bd"))
    .agg(min("CustomerID"), max("CustomerID"))
    .orderBy("Country")
    .show(10)

+---------+----------+---------------+---------------+
|  Country|        bd|min(CustomerID)|max(CustomerID)|
+---------+----------+---------------+---------------+
|Australia|1966-09-17|          12422|          12422|
|Australia|1967-11-29|          12424|          12424|
|Australia|1985-12-30|          12386|          12386|
|Australia|1986-08-28|          12434|          12434|
|Australia|1987-02-26|          12393|          12393|
|Australia|1989-08-28|          12415|          12415|
|Australia|1993-01-31|          12431|          12431|
|Australia|1993-04-02|          12388|          12388|
|  Austria|1970-08-06|          12865|          12865|
|  Austria|1973-07-16|          12818|          12818|
+---------+----------+---------------+---------------+
only showing top 10 rows



## 4 Операции над несколькими датафреймами

### Union

In [46]:
customerDf.count

res46: Long = 507L

In [47]:
customerDf.union(customerDf).count

res47: Long = 1014L

In [48]:
customerDf.union(customerDf).union(customerDf).count

res48: Long = 1521L

In [49]:
customerDf.count * 5

res49: Long = 2535L

In [50]:
(1 to 5).map(a => customerDf).reduce((x, y) => x.union(y)).count

res50: Long = 2535L

### Join

In [51]:
val retailDf = spark.read.format("json").load("data/retail_data.json")

retailDf: DataFrame = [CustomerID: string, Description: string ... 5 more fields]

In [52]:
customerDf.printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



In [53]:
retailDf.printSchema

root
 |-- CustomerID: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- UnitPrice: string (nullable = true)



In [54]:
(customerDf.dtypes.map(_._1)).toSet.intersect((retailDf.dtypes.map(_._1)).toSet)

res54: Set[String] = Set("CustomerID")

In [55]:
customerDf.join(retailDf, Seq("CustomerID")).show(10)

+----------+--------------------+-------------------+--------------+--------------------+---------------+----------------+--------------------+---------------+---------+--------+---------+---------+
|CustomerID|             Address|          Birthdate|       Country|               Email|           Name|        Username|         Description|    InvoiceDate|InvoiceNo|Quantity|StockCode|UnitPrice|
+----------+--------------------+-------------------+--------------+--------------------+---------------+----------------+--------------------+---------------+---------+--------+---------+---------+
|     12346|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|MEDIUM CERAMIC TO...|1/18/2011 10:01|   541431|   74215|    23166|     1.04|
|     12346|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|MEDIUM CERAMIC TO...|1/18/2011 10:17|  C541433|  -74215|    23166|     1.04|
|    

In [56]:
val newRetailDf = retailDf.withColumnRenamed("CustomerID", "rightCustomerID")

val customerRetailDf = customerDf.join(newRetailDf, customerDf("CustomerID") === newRetailDf("rightCustomerID"), "left")
customerRetailDf.show(10)

+--------------------+-------------------+--------------+----------+--------------------+---------------+----------------+---------------+--------------------+---------------+---------+--------+---------+---------+
|             Address|          Birthdate|       Country|CustomerID|               Email|           Name|        Username|rightCustomerID|         Description|    InvoiceDate|InvoiceNo|Quantity|StockCode|UnitPrice|
+--------------------+-------------------+--------------+----------+--------------------+---------------+----------------+---------------+--------------------+---------------+---------+--------+---------+---------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|          12346|MEDIUM CERAMIC TO...|1/18/2011 10:17|  C541433|  -74215|    23166|     1.04|
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|          12346|ME

newRetailDf: DataFrame = [rightCustomerID: string, Description: string ... 5 more fields]
customerRetailDf: DataFrame = [Address: string, Birthdate: string ... 12 more fields]

In [57]:
customerRetailDf.select("CustomerID").show(10)

+----------+
|CustomerID|
+----------+
|     12346|
|     12346|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
+----------+
only showing top 10 rows



## Stop

In [58]:
spark.stop()